# Series objects

time-independent function

$$u(\textbf{x}) \leftrightarrow \texttt{Function}$$

$$u(\textbf{x})=\sum_ju_j\xi_j(\textbf{x})$$

We denote by $\leftrightarrow$ a correspondence between mathematical notation and its implementation in `lucifex`.

time-dependent function

$$u(\textbf{x},t) \leftrightarrow \texttt{FunctionSeries}$$

$$u(\textbf{x},t)\approx\sum_ju_j(t)\xi_j(\textbf{x})$$

$$u(\textbf{x}, t\leq t^{n+1})\approx
\begin{bmatrix}
u^{n+1}(\textbf{x}) \\
u^n(\textbf{x}) \\
u^{n-1}(\textbf{x}) \\
\vdots
\end{bmatrix}$$

where $u^n(\textbf{x})=\sum_ju_j^n\xi_j(\textbf{x})$ and $u_j^n\approx u_j(t^n)$

time-dependent constant 

$$C(t) \leftrightarrow \texttt{ConstantSeries}$$

time-independent constant 

$$C \leftrightarrow \texttt{Constant}$$

time-dependent expression 

$$f(a(\textbf{x}, t), b(\textbf{x}, t)) \leftrightarrow \texttt{ExprSeries}$$

time-independent expression 

$$f \leftrightarrow \texttt{Expr}$$

In [1]:
from lucifex.fdm import FunctionSeries, FiniteDifference, DT, AB, AM
from lucifex.fem import LUCiFExConstant as Constant
from lucifex.mesh import interval_mesh


mesh = interval_mesh(1.0, 10)
order = 3

a = FunctionSeries((mesh, 'P', 1), 'a', order=order)
a.update(0.1)

b = FunctionSeries((mesh, 'P', 1), 'b', order=order)
b.update(0.3)

print('\n After setting present value:')
print(repr(a))
print(repr(b))

a.update(0.2, future=True)
b.update(0.4, future=True)
print('\n After setting future value:')
print(repr(a))
print(repr(b))

t = 0.0
a.forward(t)
b.forward(t)
print('\n After stepping forward:')
print(repr(a))
print(repr(b))

t = 1.0
a.forward(t)
b.forward(t)
print('\n After stepping forward again:')
print(repr(a))
print(repr(b))


 After setting present value:
FunctionSeries(Unsolved, Unsolved; a⁽⁰⁾; Unsolved)
FunctionSeries(Unsolved, Unsolved; b⁽⁰⁾; Unsolved)

 After setting future value:
FunctionSeries(Unsolved, Unsolved; a⁽⁰⁾; a⁽⁺¹⁾)
FunctionSeries(Unsolved, Unsolved; b⁽⁰⁾; b⁽⁺¹⁾)

 After stepping forward:
FunctionSeries(Unsolved, a⁽⁻¹⁾; a⁽⁰⁾; Unsolved)
FunctionSeries(Unsolved, b⁽⁻¹⁾; b⁽⁰⁾; Unsolved)

 After stepping forward again:
FunctionSeries(a⁽⁻²⁾, a⁽⁻¹⁾; Unsolved; Unsolved)
FunctionSeries(b⁽⁻²⁾, b⁽⁻¹⁾; Unsolved; Unsolved)


## Operations on series objects

unary operations

$$\mathcal{O}_1 \leftrightarrow \texttt{Callable[[Series], ExprSeries]}$$

$$\mathcal{O}_1(u(\textbf{x},t)) \leftrightarrow \texttt{ExprSeries}$$

$$\mathcal{O}_1(u(\textbf{x},t)) \approx 
\begin{bmatrix}
\mathcal{O}_1(u^{n+1}(\textbf{x})) \\
\mathcal{O}_1(u^n(\textbf{x})) \\
\mathcal{O}_1(u^{n-1}(\textbf{x})) \\
\vdots
\end{bmatrix}$$

binary operations

$$\mathcal{O}_2 \leftrightarrow \texttt{Callable[[Series, Series], ExprSeries]}$$

$$\mathcal{O}_2(a(\textbf{x},t), b(\textbf{x},t)) \leftrightarrow \texttt{ExprSeries}$$

$$\mathcal{O}_2(a(\textbf{x},t), b(\textbf{x},t)) \approx 
\begin{bmatrix}
\mathcal{O}_2(a^{n+1}(\textbf{x}), b^{n+1}(\textbf{x})) \\
\mathcal{O}_2(a^n(\textbf{x}), b^n(\textbf{x})) \\
\mathcal{O}_2(a^{n-1}(\textbf{x}), b^{n-1}(\textbf{x})) \\
\vdots
\end{bmatrix}$$

In [2]:
e = a * (1 + b)**2

print(type(e))
print(repr(e))

<class 'lucifex.fdm.series.ExprSeries'>
ExprSeries(a⁽⁻²⁾ * (1 + b⁽⁻²⁾) ** 2, a⁽⁻¹⁾ * (1 + b⁽⁻¹⁾) ** 2; Unsolved; Unsolved)


`lucifex.fdm.ufl_operators` provides overloaded versions of the `ufl`'s `inner`, `grad` and `curl` operators, which can accept arguments of type `Series` in addition to their usual argument types.

## Finite difference operators

In [ ]:
print('\n Time derivative:')
dt = 0.001
print(DT(a, dt))
print(DT(a))
dt = Constant(mesh, 0.001, name='dt')
print(DT(a, dt))

print('\n Adams-Bashforth explicit discretizations:')
for n in range(1, order + 1):
    print(AB(n)(a))

print('\n Adams-Moulton implicit discretizations:')
for n in range(1, order + 1):
    print(AM(n)(a))
    print(AM(n)(a, False))

print('\n User-defined discretization:')
FD_user = FiniteDifference({0: 2.0, -1: -1.0}, init=AB(1))
print(FD_user(a))


 Time derivative:
(v_1 + -1.0 * a⁽⁰⁾) / 0.001
v_1 + -1.0 * a⁽⁰⁾
(v_1 + -1.0 * a⁽⁰⁾) / dt

 Adams-Bashforth explicit discretizations:
a⁽⁰⁾
1.5 * a⁽⁰⁾ + -0.5 * a⁽⁻¹⁾
1.9166666666666667 * a⁽⁰⁾ + -1.3333333333333333 * a⁽⁻¹⁾ + 0.4166666666666667 * a⁽⁻²⁾

 Adams-Moulton implicit discretizations:
v_1
a⁽⁺¹⁾
0.5 * v_1 + 0.5 * a⁽⁰⁾
0.5 * a⁽⁺¹⁾ + 0.5 * a⁽⁰⁾
0.4166666666666667 * v_1 + 0.6666666666666666 * a⁽⁰⁾ + -0.08333333333333333 * a⁽⁻¹⁾
0.4166666666666667 * a⁽⁺¹⁾ + 0.6666666666666666 * a⁽⁰⁾ + -0.08333333333333333 * a⁽⁻¹⁾

 User-defined discretization:
2.0 * a⁽⁰⁾ + -1.0 * a⁽⁻¹⁾


: 